# Continious Optimization: Rastrigin Function

In the previous activity, we solve some discrete (in particular, binary) problems. In this activity, we use several algorithms to solve a continuous problem, i.e., the solution is encoded as an array of float numbers. 

## Problem description
The Rastrigin function is a non-convex function used as a performance test problem for optimization algorithms. It is a typical example of a non-linear multimodal function. Finding the minimum of this function is a fairly difficult problem due to its large search space and its large number of local minima.

On an $n$-dimensional domain, the function to be **minimized** is:

$f(\mathbf{x}) = An + \sum _{i=1}^{n}{x_{i}^{2}-A\cos(2\pi x_{i})}$

where $A=10$ and $x_{i}\in [-5.12,5.12]$. It has a global minimum at $\mathbf {x} =\mathbf {0}$  where $f(\mathbf {x} )=0$.

## Hill-Climbing algorithm for Rastrigin problem

Our first algorithm will be a hill-climbing algorithm. This is a simple local search technique that starts with a random solution and iteratively improves it using a neighbourhood operator. Its pseudocode is the following:

```
sol = generate_initial_solution()
while the stop criterion is not met do:
  new_sol = generate_neighbour(sol)
  if new_sol is better than sol:
    sol = new_sol
  endif
endwhile
return sol
```

For rastrigin:
* The solutions will be a list of $n$ float numbers.
* The initial solution will be randomly generated. For each value in the list, we generated a random value between -5.12 and 5.12.
* For generating a neighbour, we change a single value using a normal distribution (with mean = 0 and standard deviation = $sigma$). $sigma$ will be a parameter of our algorithm.
* We will **minimize** the rastrigin function.

In the next cell, you can see the python code for this algorithm. Analyze the code.

In [ ]:
import random, math

# ------------------------- FUNCTIONS ----------------------------------

# Generate a random solution of M floats in the range [-5.12, 5.12]
def random_solution(M):
    sol = []
    for i in range(M):
        sol.append(random.uniform(-5.12, 5.12))
    return sol

# evaluate a solution (Rastrigin function)
def evaluate_solution(s):
    fitness = 10*len(s)
    for x in s:
        fitness += (x*x - 10*math.cos(2*math.pi*x))
    return fitness

# Generate a neighbour
# - Modify on one position
# - Add a value obtained from normal distribution (0, sigma)
def get_neighbour(s, sigma):
    pos = random.randint(0, len(s)-1)
    factor = random.gauss(0, sigma)
    neighbour = s[:]
    neighbour[pos] += factor
    if neighbour[pos] < -5.12: neighbour[pos] = -5.12
    elif neighbour[pos] > 5.12: neighbour[pos] = 5.12
    return neighbour

# A simple HC
def HC(cfg):
    random.seed(cfg["seed"])
    # Generate an initial solution
    best_sol = random_solution(cfg["dimension"])
    best_fitness = evaluate_solution(best_sol)
    best_sol_iter = 0
    for i in range(1, cfg["max_iterations"]+1):
        # Generate a neighbour
        sol = get_neighbour(best_sol, cfg["sigma"])
        fitness = evaluate_solution(sol)
        # Update the best solution if better
        if fitness < best_fitness:
            best_fitness = fitness
            best_sol = sol
            best_sol_iter = i
    return best_sol, best_fitness, best_sol_iter

# --------------------------- MAIN PROGRAM ----------------------------

# Configuration
cfg = {
    "seed": 0, # seed for random numbers
    "dimension": 10, # number of variables for rastrigin
    "max_iterations": 5000, # maximum number of evaluations
    "sigma": 0.7 # Parameter for the generation of neighbours (standard deviation for normal distribution)
}

# Run the algorithm 30 times (with different seeds) ...
seeds = [50581, 61834, 22167, 64948, 71034, 80107, 78192, 7673, 56305, 64819, 55142, 37281, 69043, 52241, 78359, 40946, 47916, 69521, 37453, 63339, 85866, 35357, 73484, 37162, 86978, 93926, 98008, 38802, 3520, 1836]

sols = []
fits = []
iters = []
for seed in seeds:
    cfg["seed"] = seed
    s, f, i = HC(cfg)
    sols.append(s)
    fits.append(f)
    iters.append(i)
    
# ... and calculate statistics
hits = 0
avg_iter = 0
avg_fit = 0
avg_iter_total = 0
for i in range(len(sols)):
    avg_fit += fits[i]
    avg_iter_total += iters[i]
    if fits[i] < 0.01:
        hits += 1
        avg_iter += iters[i]
print(f"Configuration: {cfg}")
print(f"Average fitness (average iterations): {avg_fit/len(sols)} ({avg_iter_total/len(sols)})")
print(f"% hits: {hits} out of {len(sols)} ({hits*100/len(sols)}%)")
if hits > 0:
    print(f"Average number of evaluations to find the optimum: {avg_iter/hits}")

### Exercises
1. Run the algorithm with different sigma values (from 0.1 to 1.0 with step 0.1), and complete the following table.

Answer 1: (Modify the table)

|  | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| % hits | | | | | | | 33.3 | | | |
| Avg. Iters. | | | | | | | 4533.1 | | | |

2. Why do you think we obtain these results?

Answer 2: (Write your answer here)

## VNS for Rastrigin problem

VNS is a trajectory metaheuristic, i.e., it is a local search that includes some additional features to avoid local optima. Its code is quite similar to the previous one, but it uses several neighbour generation methods and it changes among them when the current method is not able to improve the solution. Its pseudocode is the following:

```
sol = generate_initial_solution()
neighbourhood = 1
while the stop criterion is not met do:
  new_sol = generate_neighbour(sol, neighbourhood)
  if new_sol is better than sol:
    sol = new_sol
    neighbourhood = 1
  else:
    neighbourhood += 1
    if neighbourhood > MAX_NEIGHBOURHOOD:
      neighbourhood = 1
    endif
  endif
endwhile
return sol
```

We will use 10 neighbourhoods. They will be the same used in HC but the different $sigma$ value, i.e., the first neighbourhood will use $sigma=0.1$, the second one will use $sigma=0.2$, and so on. 

### Exercises
1. Copy the HC code and modify it to implement the VNS technique.

In [ ]:
# VNS implementation

2. Run the algorithm and copy the results.

Answer 2: (copy the results of the algorithm)

## GA for Rastrigin problem

Finally, we will also use the GA developed in our previous lesson to solve this problem. The main changes with respect to our ONEMAX/MTTP code are:
* As fitness function (`evaluate_solution` function), you have to use the fitness function used by HC or VNS. NOTE: since VNS/HC are minimizing but GA is maximizing instead of `return fitness` use `return -fitness`.
* As the method for generating a random solution, you have to use the `random_solution` function of HC or VNS code.
* As mutation method, you can use the neighbour generation method used by HC. 
* Change the configuration of the technique (`dimension = 10`, `max_steps = 5000`) and add the $sigma$ parameter (`sigma = 0.7`).
* Finally, modify the main program to obtain similar statistical values as in HC/VNS.
* The rest of the code should not be changed.

### Exercises
1. Implement a GA for solving Rastrigin problem.

In [ ]:
# GA for Rastrigin

2. Run the algorithm with different sigma values (from 0.1 to 1.0 with step 0.1), and complete the following table.

Answer 2: (Modify the table)

|  | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| % hits | | | | | | | | | | |
| Avg. Iters. | | | | | | | | | | |

3. (Optional) Instead of using the SPX crossover operator, use the BLX-$\alpha$ one (with $\alpha = 0.5$). The BLX-$\alpha$ operator is the following:

* Given two parents, p1 and p2, it generates a single child.
* Select a random $\gamma$ value $\in$ [$-\alpha$, $\alpha$] ($\alpha = 0.5$ for our experiments).
* The child is generated as $\gamma\cdot$p1 + $(1-\gamma)\cdot$p2 (this formula should be applied to each value in the solution).

Modify the code.


In [ ]:
# GA for Rastrigin (using BLX-alpha operator)

4. (Optional) Run again the algorithm with different sigma values (from 0.1 to 1.0 with step 0.1), and complete the following table.



Answer 4: (Modify the table)

|  | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| % hits | | | | | | | | | | |
| Avg. Iters. | | | | | | | | | | |